# Aufgabe 1: Generische Rangordnungsoperatoren
Neben den linearen Filtern bzw. der Faltung stellen die *Rangordnungsoperatoren* eine weitere Klasse von Bildoperatoren dar.
Dabei wird ähnlich zur Faltung im Ortsbereich ein Fenster über das Bild geschoben.
Im Gegensatz zum linearen Filtern werden jedoch die Pixel nach Grauwerten sortiert und im Folgenden die aus dieser Ordnung resultierenden Ränge betrachtet, um das Ergebnis der Operation zu bestimmen.

Implementieren Sie die Operatoren *Minimum-*, *Maximum-* und *Medianfilter*, die für den aktuell betrachteten Pixel jeweils den ersten, letzten oder mittleren Wert der sortierten Folge setzen. Testen sie verschiedene Filtergrößen ($3 \times 3, 5 \times 5, \dots$) und interpretieren sie die Ergebnisse!
Was bewirken die Filter jeweils?

## 0. Pfade, Pakete etc.

In [1]:
import glob
import urllib.request

%matplotlib inline
import matplotlib.pyplot as plt

import imageio
import numpy as np
import math

In [ ]:
image_filter = 'Bilder/*.jpg'

## 1. Definition der Filter
Definieren Sie zunächst die Filter als Funktionen `max_filter`, `min_filter`, `median_filter`, die eine Auswahl aus einer übergebenen *sortierten* Folge von Grauwerten treffen. Können diese Filter platzsparend als Lambda-Funktionen definiert werden?

## 2. Laden des Bildes

In [ ]:
image_path = np.random.choice(glob.glob(image_filter))
image = imageio.imread(image_path)

Für diese Aufgabe ist es wichtig, das Bild im Fließkommaformat vorliegen zu haben. Andernfalls kann der Median nicht immer korrekt berechnet werden. Konvertieren sie `image` zu einer geeigneten Repräsentation:

## 3. Verrauschen des Bildes
Um ein realistisches Szenario zu simulieren, wird das Bild nun mit Salt-and-Pepper-Rauschen versehen.

Experimentieren Sie auch mit verschieden stark verrauschten Bildern!

In [ ]:
image_noisy = np.copy(image)
for _ in range(10000):
    index = tuple([np.random.randint(0,i) for i in image.shape])
    image_noisy[index] = np.random.choice([0.0,1.0])

## 4. Berechung des Filters
Definieren Sie eine Funktion `ex4_rank_filter`, die ein Bild, eine Nachbarschaftsgröße und einen Rangordnungsfilter übergeben bekommt und als Rückgabewert das gefilterte Bild liefert.

In [ ]:
def ex4_rank_filter(image, w, rank_filter):
    filtered_image = np.zeros_like(image)
    ...
    return filtered_image

Nun wird das gefilterte Bild mit Hilfe der Funktion berechnet:

In [ ]:
filtered_image = ex4_rank_filter(image_noisy, 5, median_filter)

## 5. Darstellung
Um die Wirksamkeit des Rangordnungsoperators zu überprüfen, stellen Sie `image`, `image_noisy` und `filtered_image` nebeneinander dar:

Testen sie verschiedene Filtergrößen ($3 \times 3, 5 \times 5, \dots$) und interpretieren sie die Ergebnisse!

Was bewirken die verschiedenen Rangordnungsoperatoren jeweils?